In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import LambdaLR
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier


In [ ]:
path_to_fizio = 'all_iterations_features_60_sec_clean.pkl'
path_to_folder = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/Outputs_project_data_preprocessing'

In [ ]:
dataset = pd.read_pickle(f'{path_to_folder}/6_emotions_aggregates/m_e_emo_pre_loc_sleepSegments_steps_replaced_nan_personality.pkl')


In [ ]:
dataset.columns

### Load physiology

In [ ]:
LarField_features_120_seconds = pd.read_pickle(path_to_fizio)
LarField_features_120_seconds[1][1]

In [ ]:
input_path = 'all_iterations_features_60_sec_clean.pkl'
X, y_assessments, y_labels, x_names, subjects = pd.read_pickle(input_path)
X_df = pd.DataFrame(data=X, columns=x_names)
labels = y_labels

In [ ]:
X_df = X_df.rename({'subject': 'patient_id'}, axis='columns')
X_df['samsung_BVP_HR_FROM_RR_window_1_mean'].describe()

In [ ]:
X_df.columns

In [ ]:
columns = [
           'samsung_BVP_FILT_V2_window_1_start',
       'samsung_BVP_FILT_V2_window_1_end', 'samsung_BVP_FILT_V2_window_1_mean',
       'samsung_BVP_FILT_V2_window_1_median',
       'samsung_BVP_FILT_V2_window_1_var', 'samsung_BVP_FILT_V2_window_1_std',
       'samsung_BVP_FILT_V2_window_1_min', 'samsung_BVP_FILT_V2_window_1_max',
       'samsung_BVP_FILT_V2_window_1_amplitude',
       'samsung_BVP_FILT_V2_window_1_entropy',
       'samsung_BVP_FILT_V2_window_1_percentile_25',
       'samsung_BVP_FILT_V2_window_1_percentile_50',
       'samsung_BVP_FILT_V2_window_1_percentile_75',
       'samsung_BVP_FILT_V2_window_1_iqr', 'samsung_BVP_FILT_V2_window_1_skew',
       'samsung_BVP_FILT_V2_window_1_slope',
       'samsung_BVP_FILT_V2_window_1_kurtosis',
       'samsung_BVP_FILT_V2_window_1_second_difference_mean',
       'samsung_BVP_FILT_V2_window_1_second_difference_std',
       'samsung_BVP_FILT_V2_window_1_vlf', 'samsung_BVP_FILT_V2_window_1_lf',
       'samsung_BVP_FILT_V2_window_1_hf',
       'samsung_BVP_FILT_V2_window_1_total_power',
       'samsung_BVP_FILT_V2_window_1_lh_ratio',
       'samsung_BVP_FILT_V2_window_1_peak_vlf',
       'samsung_BVP_FILT_V2_window_1_peak_lf',
       'samsung_BVP_FILT_V2_window_1_peak_hf',
       'samsung_BVP_FILT_V2_window_1_lf_nu',
       'samsung_BVP_FILT_V2_window_1_hf_nu', 'samsung_BVP_RR_window_1_start',
       'samsung_BVP_RR_window_1_end', 'samsung_BVP_RR_window_1_mean',
       'samsung_BVP_RR_window_1_median', 'samsung_BVP_RR_window_1_var',
       'samsung_BVP_RR_window_1_std', 'samsung_BVP_RR_window_1_min',
       'samsung_BVP_RR_window_1_max', 'samsung_BVP_RR_window_1_amplitude',
       'samsung_BVP_RR_window_1_entropy',
       'samsung_BVP_RR_window_1_percentile_25',
       'samsung_BVP_RR_window_1_percentile_50',
       'samsung_BVP_RR_window_1_percentile_75', 'samsung_BVP_RR_window_1_iqr',
       'samsung_BVP_RR_window_1_skew', 'samsung_BVP_RR_window_1_slope',
       'samsung_BVP_RR_window_1_kurtosis',
       'samsung_BVP_RR_window_1_second_difference_mean',
       'samsung_BVP_RR_window_1_second_difference_std',
       'samsung_BVP_RR_window_1_hr_mean_kubios',
       'samsung_BVP_RR_window_1_rmssd', 'samsung_BVP_RR_window_1_NN20',
       'samsung_BVP_RR_window_1_pNN20', 'samsung_BVP_RR_window_1_NN30',
       'samsung_BVP_RR_window_1_pNN30', 'samsung_BVP_RR_window_1_NN50',
       'samsung_BVP_RR_window_1_pNN50', 'samsung_BVP_RR_window_1_NN100',
       'samsung_BVP_RR_window_1_pNN100',
       'samsung_BVP_RR_window_1_triangular_index',
       'samsung_BVP_RR_window_1_SDSD', 'samsung_BVP_RR_window_1_CVRR',
       'samsung_BVP_RR_window_1_sd1', 'samsung_BVP_RR_window_1_sd2',
       'samsung_BVP_RR_window_1_sd1_sd2_ratio', 'samsung_BVP_RR_window_1_vlf',
       'samsung_BVP_RR_window_1_lf', 'samsung_BVP_RR_window_1_hf',
       'samsung_BVP_RR_window_1_total_power',
       'samsung_BVP_RR_window_1_lh_ratio', 'samsung_BVP_RR_window_1_peak_vlf',
       'samsung_BVP_RR_window_1_peak_lf', 'samsung_BVP_RR_window_1_peak_hf',
       'samsung_BVP_RR_window_1_lf_nu', 'samsung_BVP_RR_window_1_hf_nu',
       'samsung_BVP_HR_FROM_RR_window_1_start',
       'samsung_BVP_HR_FROM_RR_window_1_end',
       'samsung_BVP_HR_FROM_RR_window_1_mean',
       'samsung_BVP_HR_FROM_RR_window_1_median',
       'samsung_BVP_HR_FROM_RR_window_1_var',
       'samsung_BVP_HR_FROM_RR_window_1_std',
       'samsung_BVP_HR_FROM_RR_window_1_min',
       'samsung_BVP_HR_FROM_RR_window_1_max',
       'samsung_BVP_HR_FROM_RR_window_1_amplitude',
       'samsung_BVP_HR_FROM_RR_window_1_entropy',
       'samsung_BVP_HR_FROM_RR_window_1_percentile_25',
       'samsung_BVP_HR_FROM_RR_window_1_percentile_50',
       'samsung_BVP_HR_FROM_RR_window_1_percentile_75',
       'samsung_BVP_HR_FROM_RR_window_1_iqr',
       'samsung_BVP_HR_FROM_RR_window_1_skew',
       'samsung_BVP_HR_FROM_RR_window_1_slope',
       'samsung_BVP_HR_FROM_RR_window_1_kurtosis',
       'samsung_BVP_HR_FROM_RR_window_1_second_difference_mean',
       'samsung_BVP_HR_FROM_RR_window_1_second_difference_std'

    
]
for col in columns:
    # Check if the column can be converted to float
    if pd.api.types.is_object_dtype(X_df[col]):
        try:
            X_df[col] = pd.to_numeric(X_df[col], errors='coerce')
        except ValueError:
            pass

# Check and confirm the data types
print(X_df.dtypes)

In [ ]:
X_df['samsung_BVP_HR_FROM_RR_window_1_mean']

In [ ]:
list(X_df.columns)

### Load emotions

In [ ]:

results = []
for i in range(0, len(y_assessments)):
    results.append({
        'intenseEmotion': y_assessments[i]['response']['intenseEmotion'],
        'valence': y_assessments[i]['response']['valence'],
        'arousal': y_assessments[i]['response']['arousal'],
        'patient_id': y_assessments[i]['participant'],
        'emotion_tag': y_assessments[i]['emotionTimestamp'],
    })
    

In [ ]:
emotion_responses = pd.DataFrame(results)

In [ ]:
emotion_responses

In [ ]:
# Generate dateOfSleep

emotion_responses['emotion_tag'] = emotion_responses['emotion_tag'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') if '.' in x else datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

emotion_responses['date'] = pd.to_datetime(emotion_responses['emotion_tag'])
# emotion_responses['hour'] = emotion_responses['date'].dt.hour
emotion_responses['dateOfSleep'] = pd.to_datetime(emotion_responses['date']).dt.floor('D')
emotion_responses = emotion_responses.drop(['date'], axis = 1)
emotion_responses['dateOfSleep'] = emotion_responses['dateOfSleep'] + pd.Timedelta(days=1)
emotion_responses

In [ ]:
X_df['emotion_tag'] = X_df['emotion_tag'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') if '.' in x else datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

In [ ]:
# join HR + emotions
columns_to_choose_fizio = ['patient_id', 'emotion_tag',
        'samsung_BVP_HR_FROM_RR_window_1_start',
       'samsung_BVP_HR_FROM_RR_window_1_end',
       'samsung_BVP_HR_FROM_RR_window_1_mean',
       'samsung_BVP_HR_FROM_RR_window_1_median',
       'samsung_BVP_HR_FROM_RR_window_1_var',
       'samsung_BVP_HR_FROM_RR_window_1_std',
       'samsung_BVP_HR_FROM_RR_window_1_min',
       'samsung_BVP_HR_FROM_RR_window_1_max',
       'samsung_BVP_HR_FROM_RR_window_1_amplitude',
      'samsung_BVP_RR_window_1_start',
     'samsung_BVP_RR_window_1_end',
     'samsung_BVP_RR_window_1_mean',
     'samsung_BVP_RR_window_1_median',
     'samsung_BVP_RR_window_1_var',
     'samsung_BVP_RR_window_1_std',
     'samsung_BVP_RR_window_1_min',
     'samsung_BVP_RR_window_1_max',
     'samsung_BVP_RR_window_1_amplitude',
     'samsung_BVP_RR_window_1_rmssd',
   'samsung_BVP_RR_window_1_SDSD',  #odchylenie sd
 'samsung_BVP_RR_window_1_CVRR',                    ]

merged_emotion = pd.merge(emotion_responses, X_df[columns_to_choose_fizio], on=['patient_id','emotion_tag' ], how='inner')

In [ ]:
merged_emotion

In [ ]:
merged_emotion[merged_emotion.columns[10]]

In [ ]:
# merged_emotion['samsung_BVP_HR_FROM_RR_window_1_mean'] = pd.to_numeric(merged_emotion['samsung_BVP_HR_FROM_RR_window_1_mean'], errors='coerce')
merged_emotion['samsung_BVP_RR_window_1_std'].plot()

In [ ]:
merged_emotion['samsung_BVP_HR_FROM_RR_window_1_start'].plot()

In [ ]:
merged_emotion['samsung_BVP_HR_FROM_RR_window_1_mean'].plot()

In [ ]:
merged_emotion['samsung_BVP_HR_FROM_RR_window_1_var'].plot()

In [ ]:
merged_emotion[merged_emotion['samsung_BVP_HR_FROM_RR_window_1_start']>100000]

# Agregate to 1 column

In [ ]:
unique_values = {'NO': 0, 'YES': 1}
merged_emotion.replace({'intenseEmotion': unique_values}, inplace=True)
merged_emotion.describe()

In [ ]:
merged_emotion

In [ ]:
columns_to_group = [
                    'intenseEmotion',
                    'valence',       
                    'arousal',
'samsung_BVP_RR_window_1_rmssd',
'samsung_BVP_RR_window_1_mean',
'samsung_BVP_RR_window_1_median',
# 'samsung_BVP_RR_window_1_mean',
'samsung_BVP_HR_FROM_RR_window_1_mean',
'samsung_BVP_HR_FROM_RR_window_1_median'

    
 #     'samsung_BVP_HR_FROM_RR_window_1_start',
 #       'samsung_BVP_HR_FROM_RR_window_1_end',
 #       'samsung_BVP_HR_FROM_RR_window_1_mean',
 #       'samsung_BVP_HR_FROM_RR_window_1_median',
 #       'samsung_BVP_HR_FROM_RR_window_1_var',
 #       'samsung_BVP_HR_FROM_RR_window_1_std',
 #       'samsung_BVP_HR_FROM_RR_window_1_min',
 #       'samsung_BVP_HR_FROM_RR_window_1_max',
 #       'samsung_BVP_HR_FROM_RR_window_1_amplitude',
 #      'samsung_BVP_RR_window_1_start',
 #     'samsung_BVP_RR_window_1_end',
 #     'samsung_BVP_RR_window_1_mean',
 #     'samsung_BVP_RR_window_1_median',
 #     'samsung_BVP_RR_window_1_var',
 #     'samsung_BVP_RR_window_1_std',
 #     'samsung_BVP_RR_window_1_min',
 #     'samsung_BVP_RR_window_1_max',
 #     'samsung_BVP_RR_window_1_amplitude',
 #     'samsung_BVP_RR_window_1_rmssd',
 #   'samsung_BVP_RR_window_1_SDSD',  #odchylenie sd
 # 'samsung_BVP_RR_window_1_CVRR',                   

]

# merged_emotion[columns_to_group] = merged_emotion[columns_to_group].apply(pd.to_numeric)
grouped_df = merged_emotion.groupby(['patient_id', 'dateOfSleep'])[columns_to_group].apply(lambda x: x.values.tolist()).reset_index()
grouped_df

In [ ]:
# Calculate lengths of lists in the column
lengths = grouped_df[0].apply(len)
unique_lengths = lengths.unique()
# Plot histogram
plt.figure(figsize=(8, 6))
bins = np.arange(10) - 0.5
plt.hist(lengths, bins=bins, color='skyblue', edgecolor='black')
plt.xlabel('Length of Lists')

plt.ylabel('Frequency')
plt.title('Histogram of Lengths of Lists')
plt.grid(True)
plt.xticks(unique_lengths)
plt.show()

# Merge with final dataset 

In [ ]:
list(dataset.columns)

In [ ]:
# dataset = dataset.drop(['f1_f1',
#  'f1_f2',
#  'f1_f3',
#  'f2_f1',
#  'f2_f2',
#  'f2_f3',
#  'f3_f1',
#  'f3_f2',
#  'f3_f3',
#  'f4_f1',
#  'f4_f2',
#  'f4_f3',
#  'f5_f1',
#  'f5_f2',
#  'f5_f3',
#  'f6_f1',
#  'f6_f2',
#  'f6_f3',
#  'f7_f1',
#  'f7_f2',
#  'f7_f3',
#  'f8_f1',
#  'f8_f2',
#  'f8_f3',
#  'f9_f1',
#  'f9_f2',
#  'f9_f3', 'Timestamp'], axis = 1)

In [ ]:
merged_emotion_all = pd.merge(dataset, grouped_df, on=['patient_id','dateOfSleep' ], how='inner')

In [ ]:
# Calculate lengths of lists in the column
lengths = merged_emotion_all[0].apply(len)
unique_lengths = lengths.unique()
# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(lengths, bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Length of Lists')
plt.ylabel('Frequency')
plt.title('Histogram of Lengths of Lists')
plt.grid(True)
plt.xticks(unique_lengths)
plt.show()

## Expand features - expand features to pass to the model

In [ ]:
len(merged_emotion_all[0][0])

In [ ]:
merged_emotion_all[['emotion1', 'emotion2', 'emotion3', 'emotion4', 'emotion5', 'emotion6', 'emotion7', 'emotion8', 'emotion9']] = merged_emotion_all[0].apply(pd.Series)

merged_emotion_all = merged_emotion_all.drop(0, axis = 1 )

In [ ]:
merged_emotion_all

In [ ]:
# Identify columns containing lists of vectors
vector_columns = [col for col in merged_emotion_all.columns if isinstance(merged_emotion_all[col].iloc[0], list)]
vector_columns = ['emotion1', 
                  'emotion2', 
                  'emotion3', 
                  'emotion4', 
                  'emotion5', 
                  'emotion6', 
                  'emotion7', 
                  'emotion8', 
                  'emotion9'
]
# Define a function to expand the lists into separate columns
def expand_to_columns(row, prefix):
    return pd.Series(row[prefix], index=[f'{prefix}_{i}' for i in columns_to_group])

# Apply the function to expand each column containing lists of vectors
expanded_dfs = []
for column in vector_columns:
    prefix = column.split('_')[0]  # Extract prefix from column name
    expanded_df = merged_emotion_all.apply(lambda row: expand_to_columns(row, prefix), axis=1)
    expanded_dfs.append(expanded_df)

# Concatenate the expanded DataFrames along columns axis and include single value columns
result_df = pd.concat([merged_emotion_all[merged_emotion_all.columns.difference(vector_columns)], *expanded_dfs], axis=1)
display(result_df)

In [ ]:
merged_emotion

In [ ]:
merged_emotion['samsung_BVP_HR_FROM_RR_window_1_var'].astype(float).describe()

In [ ]:

# Step 2: Calculate default values for each column
default_values = {col: max(merged_emotion[col]) for col in columns_to_group}

# Step 3: Add 0.5 to these default values
adjusted_default_values = {col: val + 0.1 * val for col, val in default_values.items()}

# Step 4: Fill NaN values with the calculated values
for col in result_df.columns:
    for substring in columns_to_group:
        if substring in col:
            # print(col)  # For debugging purposes
            default_value = adjusted_default_values[substring]
            result_df[col] = result_df[col].apply(
                lambda x: default_value if isinstance(x, float) and np.isnan(x) else x
            )

# Display the adjusted default values used for filling NaN
print("Adjusted default values for each column:\n", adjusted_default_values)

In [ ]:
result_df

In [ ]:
adjusted_default_values = pd.DataFrame(adjusted_default_values, list(adjusted_default_values.keys()))

adjusted_default_values.to_pickle('values_instead_of_nan_fizio.pkl')

In [ ]:
adjusted_default_values.keys()

In [ ]:
result_df.to_pickle(f'{path_to_folder}/9_final_dataset/m_e_emo_pre_loc_sleepSegments_steps_replaced_nan_personality_fizio.pkl')

In [ ]:
default_values

In [ ]:
result_df